In [1]:
import pandas as pd

task1_data = pd.read_csv("/data/yingfei/cancer_data/llm_prompt_data/llm_inference_data/sens_simp_70b_inf.csv")
# task2_data = pd.read_csv("/data/yingfei/cancer_data/llm_prompt_data/llm_inference_data/bd_simp_70b_inf.csv")
print(task1_data.shape)
# print(task2_data.shape)

(20628, 4)


In [2]:
### Task 1
task1_data.head()

,cell_id,prompt,answer,output_70
0,ACH-000001,Think step by step and decide in a single word...,Sensitive,Reasoning:?\n\nAnswer: Sensitive.\nReasoning: ...
1,ACH-000001,Think step by step and decide in a single word...,Sensitive,Reasoning:?\n\nAnswer: Sensitive.\nReasoning: ...
2,ACH-000001,Think step by step and decide in a single word...,Sensitive,Reasoning:?\n\nAnswer: Sensitive.\nReasoning: ...
3,ACH-000001,Think step by step and decide in a single word...,Sensitive,Reasoning:?\n\nAnswer: Sensitive.\nReasoning: ...
4,ACH-000001,Think step by step and decide in a single word...,Resistant,Reasoning:?\n\nAnswer: Sensitive.\nReasoning: ...


In [3]:
task1_data = task1_data.loc[task1_data.output_70.apply(lambda x: len(str(x))>3)]
print(task1_data.shape)

(200, 4)


In [4]:
task1_data.loc[task1_data.output_70.apply(lambda x: "Drug Sensitivity:" in x)]

,cell_id,prompt,answer,output_70
34,ACH-000002,Think step by step and decide in a single word...,Sensitive,Reasoning:?\n\nAnswer: Sensitive.\nReasoning: ...
42,ACH-000002,Think step by step and decide in a single word...,Sensitive,Reasoning:?\n\nAnswer:\nDrug Sensitivity: Resi...
58,ACH-000002,Think step by step and decide in a single word...,Resistant,Reasoning:?\n\nAnswer:\nDrug Sensitivity: Resi...
63,ACH-000002,Think step by step and decide in a single word...,Resistant,Reasoning:?\n\nAnswer:\nDrug Sensitivity: Resi...
73,ACH-000004,Think step by step and decide in a single word...,Sensitive,Reasoning:?\n\nAnswer:\nDrug Sensitivity: Resi...
105,ACH-000007,Think step by step and decide in a single word...,Sensitive,Reasoning:?\n\nAnswer: Sensitive.\nReasoning: ...
114,ACH-000007,Think step by step and decide in a single word...,Sensitive,Reasoning:?\n\nAnswer: Sensitive.\nReasoning: ...
164,ACH-000008,Think step by step and decide in a single word...,Resistant,Reasoning:?\n\nAnswer:\nDrug Sensitivity: Resi...
166,ACH-000008,Think step by step and decide in a single word...,Resistant,Reasoning:?\n\nAnswer:\nDrug Sensitivity: Resi...


In [5]:
task1_data.loc[52, "output_70"]

'Answer: Resistant.\nReasoning: The drug ARA-G is an anti-metabolite that targets the mTOR pathway. The cell line has a mutation in RPTOR, which is a key component of the mTOR pathway. This mutation is likely to confer resistance to the drug, as it may disrupt the binding of the drug to its target or alter the downstream signaling pathways. Additionally, the cell line has mutations in CDKN2A, CEBPA, and KDM5C, which may also contribute to resistance to the drug by disrupting cell cycle regulation and epigenetic modulation. Therefore, the drug sensitivity of the cell line with these mutations is likely to be Resistant.'

In [6]:
# task1_data['prediction'] = task1_data.output_70.apply(lambda x: x.split("Answer:")[1].split("Reasoning:")[0][:-2][1:])
for i in range(len(task1_data)):
    output = task1_data.output_70[i]
    if "Answer: " in output:
        pred = output.split("Answer: ")[1].split("\nReasoning:")[0]
    elif "Answer:\n" in output:
        pred = output.split("Answer:\n")[1].split("\nReasoning:")[0]
    if "Drug Sensitivity: " in pred:
        pred = pred.split("Drug Sensitivity: ")[1]
    if "." in pred:
        pred = pred.replace(".", "")
    task1_data.loc[i, "prediction"] = pred

task1_data.prediction.unique() # both sensitive and resistant

array(['Sensitive', 'Resistant'], dtype=object)

In [7]:
task1_acc = sum(task1_data.prediction == task1_data.answer) / len(task1_data)
task1_acc

0.525

In [8]:
task1_data.loc[task1_data.prediction == "Resistant"]

,cell_id,prompt,answer,output_70,prediction
42,ACH-000002,Think step by step and decide in a single word...,Sensitive,Reasoning:?\n\nAnswer:\nDrug Sensitivity: Resi...,Resistant
52,ACH-000002,Think step by step and decide in a single word...,Sensitive,Answer: Resistant.\nReasoning: The drug ARA-G ...,Resistant
58,ACH-000002,Think step by step and decide in a single word...,Resistant,Reasoning:?\n\nAnswer:\nDrug Sensitivity: Resi...,Resistant
63,ACH-000002,Think step by step and decide in a single word...,Resistant,Reasoning:?\n\nAnswer:\nDrug Sensitivity: Resi...,Resistant
73,ACH-000004,Think step by step and decide in a single word...,Sensitive,Reasoning:?\n\nAnswer:\nDrug Sensitivity: Resi...,Resistant
82,ACH-000004,Think step by step and decide in a single word...,Resistant,Answer: Resistant.\nReasoning: The drug GEFITI...,Resistant
88,ACH-000004,Think step by step and decide in a single word...,Sensitive,Answer: Resistant.\nReasoning: ARA-G is a nucl...,Resistant
136,ACH-000008,Think step by step and decide in a single word...,Resistant,Answer: Resistant.\nReasoning: The drug VX-11E...,Resistant
164,ACH-000008,Think step by step and decide in a single word...,Resistant,Reasoning:?\n\nAnswer:\nDrug Sensitivity: Resi...,Resistant
166,ACH-000008,Think step by step and decide in a single word...,Resistant,Reasoning:?\n\nAnswer:\nDrug Sensitivity: Resi...,Resistant


In [9]:
task1_data['reasoning'] = task1_data.output_70.apply(lambda x: x.split("\nReasoning:")[1][1:] if x.split("\nReasoning:")[1][0] == " " else x.split("\nReasoning:")[1])
for i in range(3):
    print(task1_data.reasoning.unique()[i])

The drug VX-11E is a MEK inhibitor, which targets the ERK MAPK signaling pathway. The mutations in the cell line, NOTCH1, NOTCH3, PIK3R1, PPP2R1A, TP53, TSC2, and WHSC1L1, do not affect the ERK MAPK pathway directly. Therefore, the drug should still be effective in inhibiting the ERK pathway in this cell line, making it sensitive to the drug.
The drug AGI-6780 is an inhibitor of IDH2, which is a key enzyme in the citric acid cycle. The R140Q mutation in IDH2 has been shown to result in a gain-of-function, leading to increased production of 2-hydroxyglutarate (2HG). 2HG is a key intermediate in the citric acid cycle and its accumulation has been linked to various cellular processes, including cell proliferation and survival.

The cell line with the mutations NOTCH1, NOTCH3, PIK3R1, PPP2R1A, TP53, TSC2, and WHSC1L1 has been shown to have increased activity in the PI3K/Akt signaling pathway, which is downstream of IDH2 in the citric acid cycle. Inhibition of IDH2 by AGI-6780 would reduce 

In [10]:
print(task1_data["prompt"][0])
print(task1_data["answer"][0])
print(task1_data["prediction"][0])
task1_data['reasoning'][0]

Think step by step and decide in a single word reflecting the drug sensitivity of the drug on the cell line with given mutations: [Sensitive/Resistant], [Reasoning].
Drug and cell line mutations: 
The drug is VX-11E. The drug SMILES structure is CC1=CN=C(NC2=C(Cl)C=C(F)C=C2)N=C1C3=CNC(C(N[C@@H](C4=CC(Cl)=CC=C4)CO)=O)=C3. Drug target is ERK2. Drug target pathway is ERK MAPK signaling.
The mutations of the cell line are NOTCH1, NOTCH3, PIK3R1, PPP2R1A, TP53, TSC2, WHSC1L1.
Drug Sensitivity: ?
Sensitive
Sensitive


'The drug VX-11E is a MEK inhibitor, which targets the ERK MAPK signaling pathway. The mutations in the cell line, NOTCH1, NOTCH3, PIK3R1, PPP2R1A, TP53, TSC2, and WHSC1L1, do not affect the ERK MAPK pathway directly. Therefore, the drug should still be effective in inhibiting the ERK pathway in this cell line, making it sensitive to the drug.'

In [11]:
task1_data.head()

,cell_id,prompt,answer,output_70,prediction,reasoning
0,ACH-000001,Think step by step and decide in a single word...,Sensitive,Reasoning:?\n\nAnswer: Sensitive.\nReasoning: ...,Sensitive,"The drug VX-11E is a MEK inhibitor, which targ..."
1,ACH-000001,Think step by step and decide in a single word...,Sensitive,Reasoning:?\n\nAnswer: Sensitive.\nReasoning: ...,Sensitive,"The drug AGI-6780 is an inhibitor of IDH2, whi..."
2,ACH-000001,Think step by step and decide in a single word...,Sensitive,Reasoning:?\n\nAnswer: Sensitive.\nReasoning: ...,Sensitive,The drug AZD8931 is a pan-EGFR inhibitor that ...
3,ACH-000001,Think step by step and decide in a single word...,Sensitive,Reasoning:?\n\nAnswer: Sensitive.\nReasoning: ...,Sensitive,The drug IMATINIB is a tyrosine kinase inhibit...
4,ACH-000001,Think step by step and decide in a single word...,Resistant,Reasoning:?\n\nAnswer: Sensitive.\nReasoning: ...,Sensitive,The drug XMD11-85H is a potent and selective L...


In [13]:
from sklearn.metrics import confusion_matrix, accuracy_score
confusion_matrix(task1_data.answer, task1_data.prediction)

array([[ 6, 91],
       [ 4, 99]])

In [14]:
accuracy_score(task1_data.answer, task1_data.prediction)

0.525

In [12]:
### Result analysis
# 1) information used in the reasoning and its validity
# 2) any difference in reasoning for sensitive or resistant?
# 3) repetitive information within the reasoning and with the prompt